In [0]:
import sys
sys.path.append("../../")

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from src_staging.sv_policy_stage import create_policy_stage, sat_policy_details_schema

In [0]:
partition_name = 'partition_date'
partition_date = '20250828'

In [0]:
_bz_table = 'ctl_central_published.sc_bz_source_name.dt_customer_journey_daily'
_sat_sv_table = 'ctl_central_published.sc_sv_raw_vault.sat_policy_details'
partition_date_fmt = f"{partition_date[:4]}-{partition_date[4:6]}-{partition_date[6:]}"

In [0]:
sv_df = create_policy_stage(_bz_table, partition_name, partition_date_fmt)
sv_df.display()

In [0]:
incoming = (
    sv_df.select(
        "policy_number",
        "policy_type",
        "coverage_amount",
        "premium_amount",
        "start_date",
        "end_date",
        "record_source",
        "event_ts",
        "partition_date",
    )
    # hash only descriptive attrs (no BK/timestamps)
    .withColumn(
        "hashdiff",
        sha2(concat_ws("||",
            coalesce(col("policy_type"), lit("")),
            coalesce(col("coverage_amount").cast("string"), lit("")),
            coalesce(col("premium_amount").cast("string"), lit("")),
            coalesce(col("start_date").cast("string"), lit("")),
            coalesce(col("end_date").cast("string"), lit(""))
        ), 256)
    )
    .withColumn("effective_from_ts", col("event_ts"))
    .select("policy_number","policy_type","coverage_amount","premium_amount",
            "start_date","end_date","hashdiff","effective_from_ts",
            "record_source","partition_date")
    )

incoming.createOrReplaceTempView("incoming_policy")

In [0]:
if not spark.catalog.tableExists(_sat_sv_table):
    (
        spark.createDataFrame([], sat_policy_details_schema())
        .write.format("delta")
        .saveAsTable(_sat_sv_table)
    )

spark.sql(f"""
    MERGE INTO {_sat_sv_table} AS tgt
    USING incoming_policy AS src
    ON  tgt.policy_number  = src.policy_number 
    AND tgt.is_current = true

    WHEN MATCHED AND tgt.hashdiff <> src.hashdiff THEN
    UPDATE SET
        tgt.effective_to_ts = src.effective_from_ts,
        tgt.is_current      = false

    WHEN NOT MATCHED THEN
    INSERT (
        policy_number, policy_type, coverage_amount, premium_amount,
        start_date, end_date,
        hashdiff, effective_from_ts, effective_to_ts, is_current,
        record_source, _ingest_ts, partition_date
    )
    VALUES (
        src.policy_number, src.policy_type, src.coverage_amount, src.premium_amount,
        src.start_date, src.end_date,
        src.hashdiff, src.effective_from_ts, NULL, true,
        src.record_source, current_timestamp(), src.partition_date
    )
""")